# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686607


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/33 [00:00<?, ?it/s]

Rendering models:   6%|▌         | 2/33 [00:01<00:27,  1.13it/s]

Rendering models:   9%|▉         | 3/33 [00:05<00:53,  1.78s/it]

Rendering models:  12%|█▏        | 4/33 [00:07<00:51,  1.78s/it]

Rendering models:  15%|█▌        | 5/33 [00:07<00:36,  1.29s/it]

...

Rendering models:  91%|█████████ | 30/33 [00:11<00:00,  4.97it/s]

Rendering models:  94%|█████████▍| 31/33 [00:11<00:00,  5.54it/s]

Rendering models:  97%|█████████▋| 32/33 [00:11<00:00,  5.16it/s]

Rendering models: 100%|██████████| 33/33 [00:12<00:00,  4.90it/s]

Stargazer28211                        0.001977
framosuw                              0.000518
CThomas                               0.000321
cschwefl                              0.000250
djswanso                              0.000156
not-logged-in-ff40b895d06514783cbc    0.001593
mtajnai                               0.000219
theetick                              0.002297
sn346808                              0.024263
sn346808                              0.038557
sn346808                              0.019477
sn346808                              0.022412
ryanwong                              0.001657
not-logged-in-8abbc4e3c4658fb03367    0.018051
not-logged-in-623cb7cd00f5dbcaf5ac    0.000365
Nicola123                             0.018391
caseyroberto                          0.000178
puic                                  0.000238
Tenorclef                             0.000473
Linda_J._Berkel                       0.000175
superphoton                           0.000288
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())